In [1]:
############    Loading Python Packages
%matplotlib inline
import pymc3 as pm
import numpy as np
import scipy.stats as stats
import theano as T
import theano.tensor as tt
from theano.compile.ops import as_op
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
sb.set_context(
    "talk",
    font_scale=1,
    rc={
        "lines.linewidth": 2.5,
        "text.usetex": True,
        "font.family": 'serif',
        "font.serif": 'Palatino',
        "font.size": 16
    })
from matplotlib import rc
# rc('font', **{'family': 'sans-serif', 'sans-serif': ['Helvetica'], 'size': 16})
# ## for Palatino and other serif fonts use:
# rc('font', **{'family': 'serif', 'serif': ['Palatino'], 'size': 16})

font = {'family': 'serif', 'serif': 'Palatino', 'weight': 'bold', 'size': 16}
rc('font', **font)
rc('text', usetex=True)

Let's import the data to a pandas dataframe so that we can manipulate the data easily. 

In [19]:
##################### Reading beacon trace into a dataframe ##########################
power = ['-20db', '-15db', '-12db']
time = ['0.1', '0.5', '1']
df = pd.DataFrame({
    'Timestamp': [],  #### Time at which beacon is heard in seconds
    'Beacon': [],     #### beacon ID
    'Time_Interval': []  ##### Discrete time interval. Breaking time into discrete time windows. 
})

p = power[1]      ###reading in -15db
t = time[2]       ###reading in 1 sec. 
filename = "sortedTrace" + p + t + "sec.txt"
df_temp = pd.read_table(filename, delimiter=" ")
df_temp.columns = ['Timestamp', 'Beacon']

time_window = 10.0       ### each time window duration. 

# setting up the time interval value for each record based on time window value.
df_temp['Time_Interval'] = (df_temp['Timestamp'] / time_window).astype(int)
df = df_temp.copy()

In [20]:
################### Reading in the beacon locations ###################
a = pd.read_excel('beaconLocation.csv')
a.columns = ['Beacon','x','y']
beacX = np.array(a['x'])
beacY = np.array(a['y'])
beacX = (beacX + 40)*0.0254  ######## Adjusting co-ordinate system; meter conversion
beacY = (beacY + 80)*0.0254

In [21]:
# Using the noStack, oneStack and twoStack models based on the respective situation
def theta(frequency, power, distance, aisle):
    # no aisle (aisle==0), one aisle (aisle==1) and two aisles (aisle==2)
    # pm.categorical starts with 1

    if pm.math.eq(aisle, 0):
        z = -0.101 - 0.012 * frequency + 0.056 * power - 0.272 * distance + 0.189 * power * distance
        return (z)
    elif pm.math.eq(aisle, 1):
        z = -0.236 - 0.026 * frequency + 0.303 * power - 0.105 * distance + 0.018 * power * distance
        return (z)
    elif pm.math.eq(aisle, 2):
        z = -0.305 - 0.033 * frequency + 0.604 * power - 0.156 * distance + 0.017 * power * distance
        return (z)

In [22]:
# Fixing the power
r = np.power(
    10,
    (float(str(p).strip('db')) - float(str(power[2]).strip('db'))) /
    10.0  # reference is -12db
)
# Fixing the frequency
f = 1 / float(t) # frequency of transmission
beaconNumber = 60 # number of beacons in our set-up

# one location estimate for each time interval. 
# N is the number of places where we determine location and is controlled by time_window

N = df['Time_Interval'].max() + 1
x = np.ones(N, dtype=object)
y = np.ones(N, dtype=object)

BeaconGroups = 3

frequency = f
power = r
with pm.Model() as model:

    # speed of a person on average varies between 0.254 and 0.762 metre/second
    s = pm.Uniform("speed", 0.254, 0.762)
    # Our testbed layout is 13.2 * 7.1 metres  
    x[0] = pm.Uniform("x_0",0,13.2)     ####### in metres. 
    y[0] = pm.Uniform("y_0",0, 7.1)

    for i in range(1, N):
        # Subsequent Locations will lie in a circle based on previous location as a center.
        x[i] = pm.Normal('x_%i' % i, mu=x[i-1], sd=s*time_window)
        y[i] = pm.Normal('y_%i' % i, mu=y[i-1], sd=s*time_window)
    

    # There are three groups iot beacons in our case: [0-11] [12-35] [36-59]
    # regardless of where the person is, the same noise model
    # (no stack, one stack or two) will apply to the entire subgroup)       
    
    Stack = T.shared(np.array([[0, 1, 2], [1, 0, 1], [2, 1, 0]]))

       
    # loop over locations
    for j in range(N):
        
        # based on the geometry, the models switch based on y
        # finding in which aisle the person currently is
        
        aisle = pm.math.switch(y[j] <= 2.032, 0, 1)
        aisle = pm.math.switch(y[j]>=4.0894, 2, aisle) 
        
        data = df[(df.Time_Interval==j)]    ### reading data for i-th interval
        beacon = np.array(data['Beacon'])
        timeStamp = np.array(data['Timestamp'])
        startTime = timeStamp[0]
        endTime = timeStamp[timeStamp.size - 1]
        pktRecvd = []
        pktSent = []
        beaconId = []
        for i in range(beaconNumber):
            pktNo = beacon.tolist().count(i+1)
            ### eliminating beacons from whom no packet received  and a few misplaced ones.
            if pktNo != 0 and i != 8 and i != 54 and i%4 == 0:  
                beaconId.append(i)
                sentPKT = int((time_window)/float(t))
                if pktNo > sentPKT:
                    pktNo = sentPKT
                pktRecvd.append(pktNo)
                pktSent.append(sentPKT)
        packetSent = np.asarray(pktSent)
        packetRecvd = np.asarray(pktRecvd)
        
        subBeacX = []
        subBeacY = []
        ### creating location list for beacons present in the current interval
        for i in range(len(beaconId)):
            subBeacX.append(beacX[beaconId[i]])
            subBeacY.append(beacY[beaconId[i]])
        occBeacX = np.asarray(subBeacX)
        occBeacY = np.asarray(subBeacY)
        
        distance = pm.math.sqrt(
            pm.math.sqr(occBeacX - x[j]) + pm.math.sqr(occBeacY - y[j]))

        #define a probability vector for each location
        prob = tt.zeros(len(beaconId))

        for i in range(len(beaconId)):
            #get the aisle for the beacon

            BeaconGroup = int(np.ceil((beaconId[i] + 0.5) / 12) / 2)
            NumStacks = Stack[aisle, BeaconGroup]
 
            # at this point, we know the Number of stacks for beacon i

            z = pm.math.exp(
                theta(
                    frequency=frequency,
                    power=power,
                    distance=distance[i],
                    aisle=NumStacks))
            prob = tt.set_subtensor(prob[i], z)
        
        Received = pm.Binomial(
            "Data {0}".format(j),
            n=packetSent,
            p=prob,
            observed=packetRecvd)


    step = pm.Metropolis()
    trace = pm.sample(10000, step=step)
    burned_trace = trace[500:] 

/Users/subham/miniconda3/lib/python3.5/site-packages/theano/tensor/basic.py:2146: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"
/Users/subham/miniconda3/lib/python3.5/site-packages/theano/tensor/basic.py:2146: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"
100%|██████████| 10000/10000 [01:57<00:00, 84.83it/s]


In [6]:
def check(lowY, highY, posY):
    if posY >= lowY and posY<=highY:
        diff1 = posY - lowY
        diff2 = highY - posY
        if diff1 > diff2:
            posY = highY
        else:
            posY = lowY
    return posY

########### finds the actual ground truth location. 
### takes in startTime and endTime of a time interval
### returns the ground truth location co-ordinates. 
### We know the stop location co-ordinates and we know the amount of time we stopped for each location. 
### We use a simple interpolation to find locations at intermediate times (assuming average movement speed)
### stopTime is the time for which we stop at each location, maxTime is the maximum time value in a particular trace
def findActualLocation(startTime , endTime, stopTime, maxTime):
    personLocation = pd.read_table('standingLocation.csv', delimiter="\t")
    personLocation.columns = ['x', 'y']
    personLocation['x'] = personLocation['x'] + 40
    personLocation['y'] = personLocation['y'] + 80
    personLocation['x'] = personLocation['x'] * 0.0254 ### changing to meters
    personLocation['y'] = personLocation['y'] * 0.0254
    distance = 0
    time = maxTime
    for index, row in personLocation.iterrows():
        if index != 0:
            distance = distance + np.sqrt((row['x'] - X) * (row['x'] - X) + (row['y'] - Y) * (row['y'] - Y))
        time = time - stopTime
        X = row['x']
        Y = row['y']
    movingSpeed = distance/time
    
    locationAtTime = endTime
    time = 0
    for index, row in personLocation.iterrows():
        if index != 0:
            d = np.sqrt((row['x'] - X) * (row['x'] - X) + (row['y'] - Y) * (row['y'] - Y))
            prevTime = time
            time = time + (d/movingSpeed)
            if time >= locationAtTime:
                dirX = row['x'] - X
                dirY = row['y'] - Y
                posY = (Y + (dirY * movingSpeed * (locationAtTime - prevTime) / d))
                ## starting and ending Y co-ordinates of three aisles.
                posY = check(lowY=2.032,highY=2.5654,posY=posY)
                posY = check(lowY=3.556,highY=4.0894,posY=posY)
                posY = check(lowY=5.08,highY=5.6134,posY=posY)
                return (X + (dirX * movingSpeed * (locationAtTime - prevTime) / d)) , posY
        time = time + stopTime
        if time >= locationAtTime:
            return row['x'], row['y']
        X = row['x']
        Y = row['y']
    return X,Y

In [23]:
maxTime = df['Timestamp'].max()
err = 0

stop_time = 10.0  ### time interval we stopped at each location
strFile1 = 'predictedLocation' + p + t + "sec.txt"
strFile2 = 'actualLocation' + p + t + "sec.txt"
outFile1 = open(strFile1, 'w')
outFile2 = open(strFile2, 'w')
for i in range(0, N):
    x,y = findActualLocation(startTime=time_window*i , endTime=time_window*(i+1), stopTime=stop_time, maxTime=maxTime)
    print('Actual Location at time' + str(i) + ' (' + str(x) + ',' + str(y) + ')')
    predX = (burned_trace['x_%i'%i]).mean()      ### predicted location at the i-th interval. 
    predY = (burned_trace['y_%i'%i]).mean()
    sdX = (burned_trace['x_%i'%i]).std()
    sdY = (burned_trace['y_%i'%i]).std()
    print('Predicted Location at time ' + str(i) + ' ('  + str(predX) + ',' + str(predY) + ')')
    error = np.sqrt((predX - x) * (predX - x) + (predY - y) * (predY - y))
    print ('Error at time ' + str(i) +  ' ' + str(error))

    if y<=5.6134:
        err = err + error
        j=i
    outFile1.write(str(i) + ' ' + str(predX) + ' ' + str(predY)+ ' ' + str(sdX) + ' ' + str(sdY) + '\n')
    outFile2.write(str(i) + ' ' + str(x) + ' ' + str(y) + '\n')

err = err / (j+1)
outFile1.close()
outFile2.close()
print("Average estimation error is " + str(err))

Actual Location at time0 (4.2164,1.524)
Predicted Location at time 0 (3.17260979901,3.26516831494)
Error at time 0 2.03006529073
Actual Location at time1 (7.04274859294,1.524)
Predicted Location at time 1 (5.3114551478,0.712617541591)
Error at time 1 1.91199332818
Actual Location at time2 (7.874,1.524)
Predicted Location at time 2 (7.7020319901,2.70481521447)
Error at time 2 1.19327179099
Actual Location at time3 (9.84253327008,1.84848350606)
Predicted Location at time 3 (8.87263801919,4.09188420371)
Error at time 3 2.44408336354
Actual Location at time4 (12.4968,2.286)
Predicted Location at time 4 (11.4521484192,3.6637131408)
Error at time 4 1.72898537401
Actual Location at time5 (12.3633241876,2.5654)
Predicted Location at time 5 (9.56089024537,4.58808335853)
Error at time 5 3.4561371456
Actual Location at time6 (9.59481704854,2.88229786674)
Predicted Location at time 6 (9.92026525756,4.06526345253)
Error at time 6 1.2269165065
Actual Location at time7 (8.7884,3.048)
Predicted Locati